<a href="https://colab.research.google.com/github/CPT-Dawn/College-Lab-Work-CSET419-Introduction-to-Generative-Artificial-Intelligence/blob/main/CSET_419_Lab_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import os
import time

# ==========================================
# STEP 1: INPUT PARAMETERS [cite: 14-21]
# ==========================================
def get_user_inputs():
    print("--- GAN Configuration ---")
    # We use defaults for smoother execution, but allow overrides
    choice = input("Dataset (mnist/fashion) [default: mnist]: ").strip().lower() or 'mnist'
    epochs = int(input("Epochs [default: 50]: ").strip() or 50)
    batch_size = int(input("Batch Size [default: 128]: ").strip() or 128)
    noise_dim = int(input("Noise Dimension [default: 100]: ").strip() or 100)
    # Learning rate is usually fixed in code for optimizers, but we can set it here
    lr = 0.0002
    save_interval = int(input("Save Interval (epochs) [default: 5]: ").strip() or 5)

    return choice, epochs, batch_size, noise_dim, lr, save_interval

# Get configuration
DATASET_CHOICE, EPOCHS, BATCH_SIZE, NOISE_DIM, LR, SAVE_INTERVAL = get_user_inputs()

# ==========================================
# STEP 2: DATASET LOADING & PREPROCESSING [cite: 9-13, 30]
# ==========================================
def load_data(choice):
    if choice == 'fashion':
        (x_train, y_train), (_, _) = tf.keras.datasets.fashion_mnist.load_data()
    else:
        (x_train, y_train), (_, _) = tf.keras.datasets.mnist.load_data()

    # Normalize images to [-1, 1] as required for GANs with Tanh output
    x_train = x_train.reshape(x_train.shape[0], 28, 28, 1).astype('float32')
    x_train = (x_train - 127.5) / 127.5

    # Batch and shuffle the data
    dataset = tf.data.Dataset.from_tensor_slices(x_train).shuffle(60000).batch(BATCH_SIZE)
    return dataset, (x_train, y_train) # Return raw data for classifier training later

train_dataset, (raw_x, raw_y) = load_data(DATASET_CHOICE)

# ==========================================
# STEP 3: BUILD GENERATOR
# ==========================================
def make_generator_model():
    model = tf.keras.Sequential()
    # Start with a dense layer that takes the noise vector
    model.add(layers.Dense(7*7*256, use_bias=False, input_shape=(NOISE_DIM,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    # Reshape into a 3D volume (7x7x256)
    model.add(layers.Reshape((7, 7, 256)))

    # Upsample to 14x14
    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    # Upsample to 14x14
    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    # Output layer: Upsample to 28x28.
    # Tanh activation is used to map pixel values to [-1, 1]
    model.add(layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))

    return model

generator = make_generator_model()

# ==========================================
# STEP 4: BUILD DISCRIMINATOR
# ==========================================
def make_discriminator_model():
    model = tf.keras.Sequential()

    # Convolutional layers to classify image
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[28, 28, 1]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    # Flatten and output a single score (Real vs Fake)
    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model

discriminator = make_discriminator_model()

# ==========================================
# STEP 5: LOSS & OPTIMIZERS [cite: 32]
# ==========================================
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    # Loss for real images (should be 1)
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    # Loss for fake images (should be 0)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    # Generator wants discriminator to think fake images are real (1)
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer = tf.keras.optimizers.Adam(LR)
discriminator_optimizer = tf.keras.optimizers.Adam(LR)

# ==========================================
# STEP 6: TRAINING LOOP [cite: 26, 36-38]
# ==========================================
# Create directories for outputs [cite: 40, 43]
os.makedirs('generated_samples', exist_ok=True)
os.makedirs('final_generated_images', exist_ok=True)

@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, NOISE_DIM])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        # Generate images
        generated_images = generator(noise, training=True)

        # Discriminator judges real and fake images
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        # Calculate losses
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    # Apply gradients
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    # Calculate simple accuracy for logging
    # Real is predicted as real if output > 0, Fake as fake if output < 0
    real_acc = tf.reduce_mean(tf.cast(real_output > 0, tf.float32))
    fake_acc = tf.reduce_mean(tf.cast(fake_output < 0, tf.float32))
    disc_acc = (real_acc + fake_acc) * 0.5

    return gen_loss, disc_loss, disc_acc

print("Starting Training...")

for epoch in range(EPOCHS):
    start = time.time()

    g_loss_metric = 0
    d_loss_metric = 0
    d_acc_metric = 0
    batches = 0

    for image_batch in train_dataset:
        g_loss, d_loss, d_acc = train_step(image_batch)
        g_loss_metric += g_loss
        d_loss_metric += d_loss
        d_acc_metric += d_acc
        batches += 1

    # Average metrics for the epoch
    g_loss_avg = g_loss_metric / batches
    d_loss_avg = d_loss_metric / batches
    d_acc_avg = d_acc_metric / batches

    # OUTPUT 1: Training Logs [cite: 37-38]
    print(f'Epoch {epoch + 1}/{EPOCHS} | D_loss: {d_loss_avg:.2f} | D_acc: {d_acc_avg*100:.2f}% | G_loss: {g_loss_avg:.2f}')

    # OUTPUT 2: Periodic Saving [cite: 27, 41]
    if (epoch + 1) % SAVE_INTERVAL == 0:
        noise = tf.random.normal([25, NOISE_DIM])
        generated_images = generator(noise, training=False)

        plt.figure(figsize=(5, 5))
        for i in range(25):
            plt.subplot(5, 5, i+1)
            plt.imshow(generated_images[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
            plt.axis('off')
        plt.savefig(f'generated_samples/epoch_{epoch+1:02d}.png')
        plt.close()

# ==========================================
# STEP 7: FINAL GENERATION & EVALUATION [cite: 42-45]
# ==========================================
print("Training Complete. Generating Final Images...")

# Generate 100 images
noise = tf.random.normal([100, NOISE_DIM])
final_images = generator(noise, training=False)

# Save the 100 images
for i in range(100):
    img = final_images[i, :, :, 0] * 127.5 + 127.5
    tf.keras.utils.save_img(f'final_generated_images/image_{i}.png', np.expand_dims(img, axis=-1))

# Helper: Create a simple "Pre-trained" classifier since we don't have an external file
# In a real scenario, you would load_model('my_classifier.h5')
print("Training a temporary classifier for evaluation (Transfer Learning simulation)...")
classifier = tf.keras.Sequential([
    layers.Flatten(input_shape=(28, 28, 1)),
    layers.Dense(128, activation='relu'),
    layers.Dense(10)
])
classifier.compile(optimizer='adam',
                   loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                   metrics=['accuracy'])
# Train quickly on the real data we loaded earlier
classifier.fit(raw_x, raw_y, epochs=1, verbose=0)

# OUTPUT 4: Predict Labels [cite: 44-45]
predictions = classifier.predict(final_images)
predicted_labels = np.argmax(predictions, axis=1)

print("\n--- Evaluation Report ---")
print("Label Distribution of Generated Images:")
unique, counts = np.unique(predicted_labels, return_counts=True)
for u, c in zip(unique, counts):
    print(f"Class {u}: {c} images")

print("\nLab completed successfully!")